## Survival Analysis for Calving Time Prediction

Data were collected using 30 prepartum dairy cow from February 2018 through April 2018. The 3 axis accelerometer was attached to automatically collect number of steps, type of activity (stop or walking), acceleration data, and other information such as temperature, pressure, etc. as raw data. The preparation of calving data are explained in the "preparation.ipynb" file. 



In [1]:
import warnings
warnings.filterwarnings('ignore')

# Import required library (link of installation written on README)
%matplotlib inline
import pandas as pd
import datetime 
import matplotlib.pyplot as plt # Import matplotlib library to make plot
from sksurv.nonparametric import kaplan_meier_estimator #import Kaplan-Meier estimator from scikit-survival library

# Import training data (please download from README)
data_x  = pd.read_csv('./Desktop/survivals.csv')
data_y = pd.read_csv('./Documents/hours.csv').to_records(index=False) # converted dataframe to numpy structured arrays


# Survival function using the hours to calve and survival status of each cow
time, survival_prob = kaplan_meier_estimator(data_y["status"], data_y["hourstocalve"])
plt.step(time, survival_prob, where="post", linewidth=4.0)
plt.ylabel("est. probability of calving $\hat{S}(t)$")
plt.xlabel("time $t$ to calve")
plt.title("Survival function regarding time to calve and survival status")


from sksurv.linear_model import CoxPHSurvivalAnalysis

estimator = CoxPHSurvivalAnalysis()
estimator.fit(data_x, data_y)

### Input Test Data for Calving Prediction

# Create sample data (test data) 'x' contains numeric value of each feature
# Insert value of activity feature

x = [(260, 1450, 1.308)]
labels = ['totalstep', 'totaltransition', 'acc']
df = pd.DataFrame.from_records(x, columns=labels)


# Plot the survival function estimation
pred_surv = estimator.predict_survival_function(df)
for i, c in enumerate(pred_surv):
    plt.step(c.x, c.y, where="post", linewidth=4.0, label="Cow no. %d" % (i + 1))
plt.ylabel("est. probability of calving $\hat{S}(t)$")
plt.xlabel("time $t$ to calve")
plt.legend(loc="right")
plt.title("Calving probability estimation using test data")

df = pd.DataFrame(c.y, c.x, columns=['prob'])
df['time']=pd.to_timedelta(df.index,unit='h')

# Insert time of data input if specific date is wanted
i = datetime.datetime(2018,4, 25, 11,0) 

# or if use current date of input data
# i = datetime.datetime.now()

# The expected probability of calving, INSERT EXPECTED PROBABILITY HERE
exprob = 0.7

df.loc[((df['prob'] - exprob).abs().idxmin()),'time'] + i

# Import historical data of activity of cow, 
# usually from 48 hours before estimated calving day 

with open('./Documents/cow15.csv', encoding='utf-8-sig') as file:
    reader = csv.reader(file, skipinitialspace=True)
    x = [[float(row[0]), float(row[1]), float(row[2])] for row in reader]

labels = ['totalstep', 'totaltransition', 'acc']
df = pd.DataFrame.from_records(x, columns=labels)

pred_surv = estimator.predict_survival_function(df)
df = pd.DataFrame(c.y, c.x, columns=['prob'])
df['time']=pd.to_timedelta(df.index,unit='h')

# The time of input data (this is from available information of calving data)
i = datetime.datetime(2018,4, 25, 11,0) 
#i = datetime.datetime.now()

# The expected probability of calving, assume the recommended confidence is 70%
exprob = 0.7

# Show predicted calving time with confidence 70%
df.loc[((df['prob'] - exprob).abs().idxmin()),'time'] + i

### Measuring the Performance of Survival Models

from sksurv.metrics import concordance_index_censored

prediction = estimator.predict(data_x)
result = concordance_index_censored(data_y["status"], data_y["hourstocalve"], prediction)
result[0]


estimator.score(data_x, data_y)

### Which Variable is Most Predictive?
import numpy as np

def fit_and_score_features(X, y):
    n_features = X.shape[1]
    scores = np.empty(n_features)
    m = CoxPHSurvivalAnalysis()
    for j in range(n_features):
        Xj = X[:, j:j+1]
        m.fit(Xj, y)
        scores[j] = m.score(Xj, y)
    return scores

scores = fit_and_score_features(data_x.values, data_y)
pd.Series(scores, index=data_x.columns).sort_values(ascending=False)
